In [1]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 37.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [4]:
import numpy as np
data = pd.read_csv('result_597.txt', sep=',', header=None, names=['state', 'year', 'score', 'number'])
# Filter out data points with 'number' less than 50
data = data[data['number'] >= 50]

data['panic_level'] = np.log(data['score'] / data['number'] +1)
data['new_score'] = data['panic_level'] 

In [69]:
# New panic_score data for Alaska
xx = [23.2369414567947410, 39.521445989608765101, 53.7055667638778710, 58.36730605363846,7.687720000743866101 * 4+ 0.003826797008514404310,  25.8558760285377510, -11.79382592439651510, -5.93822097778320310, 2.158977568149566710, 7.58016651868820210, -5.7847025394439710, 3.49432802200317410, (6.38201344013214110+ 24.9101712703704832)/2, 71.7858994007110610, 63.78337705135345510, 65.7723408341407810, 65.4892883300781210]
xx= [a/101 for a in xx]
# Years for the new data
years = [year for year in range(1920, 1937)]

# Create a new DataFrame with the given values and years for Alaska
new_alaska_data = pd.DataFrame({'state': 'Alaska', 'year': years, 'panic_score': xx})

# Calculate the new_score for the new Alaska data
new_alaska_data['new_score'] = np.log(new_alaska_data['panic_score'] + 1)

# Remove the original Alaska data from the dataset
data = data[data['state'] != 'Alaska']

# Append the new Alaska data to the dataset
data = data.append(new_alaska_data, ignore_index=True)


<ipython-input-69-ccc079c7c61e>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(new_alaska_data, ignore_index=True)


In [5]:
#Calculate the global minimum and maximum panic levels:
min_panic_level = data['new_score'].min()
max_panic_level = data['new_score'].max()
avg_panic_level = data['new_score'].mean()
print(avg_panic_level)
min_new_score, max_new_score = min_panic_level, max_panic_level

print(min_new_score, max_new_score )

0.3447897902720991
-0.147527693622174 0.5888761058286769


In [6]:
shapefile_path = '/content/ne_50m_admin_1_states_provinces.shp'
usa_map = gpd.read_file(shapefile_path)

# Filter the map to keep only the United States
usa_map = usa_map[usa_map['admin'] == 'United States of America']


In [10]:
import matplotlib.colors as mcolors


In [11]:
def normalize(value, min_value, max_value):
    return (value - min_value) / (max_value - min_value)


def create_custom_colormap(min_value, max_value, ave_value):
    colormap = mcolors.LinearSegmentedColormap.from_list(
        'custom_colormap',
        [
            (normalize(min_value, min_value, max_value), 'red'),
            (normalize(0.15, min_value, max_value), 'white'),
            (normalize(max_value, min_value, max_value), 'blue')
        ]
    )
    return colormap


In [8]:
# def create_heatmap(year, data, usa_map, output_folder):
#     data_year = data[data['year'] == year]
#     merged_map = usa_map.set_index('name').join(data_year.set_index('state'))
#     merged_map = merged_map.reset_index()

#     fig, ax = plt.subplots(1, figsize=(20, 12))
#     divider = make_axes_locatable(ax)
#     cax = divider.append_axes("right", size="5%", pad=0.1)
#     merged_map.plot(column='panic_level', cmap='coolwarm_r', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True, cax=cax)

#     ax.set_title(f"Heatmap of Panic Spread Across America's States ({year})", fontsize=20)
#     ax.set_axis_off()

#     plt.savefig(f"{output_folder}/heatmap_{year}.png", dpi=300)
#     plt.close()

# def create_heatmap(year, data, usa_map, output_folder, vmin, vmax):
#     data_year = data[data['year'] == year]
#     merged_map = usa_map.set_index('name').join(data_year.set_index('state'))
#     merged_map = merged_map.reset_index()

#     fig, ax = plt.subplots(1, figsize=(20, 12))
#     divider = make_axes_locatable(ax)
#     cax = divider.append_axes("right", size="5%", pad=0.1)
#     merged_map.plot(column='panic_level', cmap='coolwarm_r', linewidth=0.8, ax=ax, edgecolor='0.8',
#                     legend=True, cax=cax, vmin=vmin, vmax=vmax)

#     ax.set_title(f"Heatmap of Panic Spread Across America's States ({year})", fontsize=20)
#     ax.set_axis_off()

#     plt.savefig(f"{output_folder}/heatmap_{year}.png", dpi=300)
#     plt.close()


def create_heatmap(year, data, usa_map, output_folder, vmin, vmax, colormap):
    data_year = data[data['year'] == year]
    merged_map = usa_map.set_index('name').join(data_year.set_index('state'))
    merged_map = merged_map.reset_index()

    fig, ax = plt.subplots(1, figsize=(20, 12))
    
    # Set a constant aspect ratio for the Axes object
    ax.set_aspect('equal', 'box')
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    merged_map.plot(column='new_score', cmap=colormap, linewidth=0.8, ax=ax, edgecolor='0.8',
                    legend=True, cax=cax, vmin=vmin, vmax=vmax)

    ax.set_title(f"Heatmap of Panic Spread Across America's States ({year})", fontsize=20)
    ax.set_axis_off()

    plt.savefig(f"{output_folder}/heatmap_{year}.png", dpi=300)
    plt.close()


In [12]:
import os
output_folder = "heatmaps"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

unique_years = sorted(data['year'].unique())

# for year in unique_years:
#     create_heatmap(year, data, usa_map, output_folder)

# for year in unique_years:
#     create_heatmap(year, data, usa_map, output_folder, min_panic_level, max_panic_level)
colormap = create_custom_colormap(min_new_score, max_new_score, avg_panic_level)


for year in unique_years:
    create_heatmap(year, data, usa_map, output_folder, min_new_score, max_new_score, colormap)



In [13]:
import cv2
import os

video_name = 'heatmap_evolution.mp4'
frame_rate = 1  # Set the number of frames per second

# Get the dimensions of the first image
first_image = cv2.imread(f"{output_folder}/heatmap_{unique_years[0]}.png")
height, width, _ = first_image.shape

# Set up the video writer with the mp4 codec
video_writer = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (width, height))

# Add each image to the video
for year in unique_years:
    image = cv2.imread(f"{output_folder}/heatmap_{year}.png")
    video_writer.write(image)

# Release the video writer
video_writer.release()

